In [1]:
import numpy as np
import cvxpy as cp
import pandas as pd
import math
import kagglehub
import os
from GraphUtils import *
from StateObserver import *
from ReferenceConverter import *
from FeedbackController import *
from TorqueController import *

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100) 

MU = 2.0
SIGMA = 0.5

In [ ]:
""" EG_ = {
    'y_arrays': [EG],
    'x_arrays': None,
    'title': 'Generated energy',
    'xname': None,
    'yname': 'Valor',
    'legend_labels': None
}
ED_ = {
    'y_arrays': [ED],
    'x_arrays': None,
    'title': 'Dissipated energy',
    'xname': None,
    'yname': 'Valor',
    'legend_labels': None
}
K_ = {
    'y_arrays': [K],
    'x_arrays': None,
    'title': r'$K_{mppt}$ gain',
    'xname': None,
    'yname': 'Valor',
    'legend_labels': None
}
all_plots = [ED_,EG_,K_]
MultiPlot(all_plots, rows=1, cols=3, pltly=False, main_title='Análise de Funções Diversas (Matplotlib)', fig_size=(12,4))
 """

In [2]:
path = 'T1.csv'
df = pd.read_csv(path)  
v = df.iloc[:,2].values + 1e-16
PlotSingle(y=v,w=8,yname=r'Wind speed (m/s)', xname='Sample', title='Wind Turbine Dataset',pltly=True)

In [3]:
params = {
    "Bdt": 755.49,
    "Kdt": 2.7*1e9,
    "Jg": 55*1e6,
    "Jr": 55*1e6,
    "rho": 1.225,
    "Ar": math.pi * (57.5**2),
    "Rr":  57.5,
    "Cp_Max": 0.468,
    "Lambda_opt": 7.6,
    "Dmax": 1000,
    "Vmax": 5,
    "Vmin": 0.2
}

In [4]:
torque_cntrl = TorqueController(**params)
ED, EG, K = [np.array([]) for i in range(3)]
for i in range(len(v[:10000])):
    torque_cntrl.update(v[i],u_k=0,dt=0.001, **params)
    ED = np.append(ED,torque_cntrl.ED)
    EG = np.append(EG,torque_cntrl.EG)
    K = np.append(K,torque_cntrl.K_mppt)


In [24]:
i = 0
torque_cntrl = TorqueController(**params)
state_obsrv = StateObserver(TS=0.001, **params)
ref_converter = ReferenceConverter(RUL_ref=1e6)
state_obsrv.solve_lmi_gain()

In [14]:
ref_converter.show_attributes()

Reference Converter parameters:
   k    rul_ref  D_max  beta_ref
0  1  1000000.0   1000     0.001


In [ ]:
ED_k = torque_cntrl.update(v[i],dt=0.001, **params)

D_hat, beta_hat = state_obsrv.update(y_k=ED_k)

print('D_hat:',D_hat, 'beta_hat:',beta_hat)

beta_ref = ref_converter.update(D_hat)



D_hat: [1.07221277] beta_hat: [0.00105528]
Reference Converter parameters:
   k   rul_ref  D_max                 beta_ref
0  2  999999.0   1000  [0.0009989287861579324]
